# AWS CloudFormation Templates

## 클라우드포메이션 템플릿의 섹션
- Format Version(Optional) : 템플릿 버전 명시
- Description(Optional) : 템플릿에 대한 설명
- Metadata(Optional) : 템플릿에 대한 추가 데이터 명시
- Parameters(Optional) : 템플릿을 프로비전하는 시점에 논리적 리소스에 전달할 값을 정의
- Rules(Optional) : 파라미터를 검증하는 정책 명시
- Mappings(Optional) : 추가적으로 지정한 변수 MAP으로 파라미터처럼 활용
- Conditions(Optional) : 템플릿 프로비전 시 조건을 사용할 때 활용
- Transform(Optional) : 템플릿에서 사용하는 매크로 등을 정의
- Resources(필수) : 템플릿에서 프로비전할 리소스를 정의
- Outputs(Optional) : 템플릿 프로비전 이후 추가적으로 명시할 내용을 정의. 추후 다른 스택에서 참조 가능

### Format Version / Description 섹션
- CloudFormation의 버전과 설명을 명시하는 섹션
- Description 섹션은 반드시 Format Version 섹션 바로 뒤에 위치 필요

### Resources 섹션
- 스택에서 실제로 프로비전할 리소스를 정의하는 유일한 필수 섹션
- 리소스의 구성
    - Logical Name : 스택 안에서 리소스를 구분하기 위한 아이디
        - 실제 리소스 이름/아이디랑은 다른 개념으로 스택 안에서만 통용(일종의 반 번호)
    - 유형 : 리소스의 유형 (예: S3, EC2, Elastic IP 등)
        - AWS::ProductIdentifier:ResourceType 형식 (예: AWS::EC2::Instance)
    - 리소스 속성(Resource Attribute) : 리소스 자체의 공통적인 속성을 정의
        - 삭제, 업데이트, 리소스의 전후 관계등을 정의
    - 속성(Property) : 리소스의 유형별로 자세한 속성을 정의
        - 예시 : EC2의 속성 -> EC2 인스턴스 유형, AMI, 보안그룹, EBS설정, 태그(이름) 등
        - 다른 리소스 혹은 받은 파라미터를 참조하여 설정 가능

### Parameters
- 템플릿을 커스터마이징 하기 위해 설정 가능한 값
    - 일반 파라미터 : 템플릿 프로비전 시 사용자에게 받는 값. Parameter Section에서 정의
        - 예 : EC2 인스턴스 타입, AMI ID, CloudFront DNS, RDS DB Password 등
        - 활용 예시 : 하나의 정형화된 아키텍처를 찍어낼 때 DNS만 교체 혹은 인스턴스 타입만 교체
    - Pseuedo Parameter : CloudFormation에서 템플릿 프로비전 시 지정해주는 값. 프로비전 당시의 상황을 반영하여 스택으로 자동 전달
        - 예 : Region, Account ID, StackName
        - 활용 예시 : S3 버킷 이름 끝에 Account ID를 붙이고 싶은 경우

### Parameter Section
- 템플릿에서 일반 파라미터를 정의하는 섹션
- 형식
    - Logical ID : 스택 내에서만 통용되는 파라미터의 이름
    - 타입 : String, Number, List<Number>등 + AWS 지정 타입으로 AWS 리소스 지정 가능(예: Subnet, EC2 Keypair, AMI ID등)
    - 설명 : 파라미터의 설명
    - Default : 기본값
    - Allowed Value : 허용 가능한 값
    - 기타 : 파라미터 조건(길이, 정규식 등), NoEcho(암호 등에 활용)
- !Ref Intrinsic Function으로 참조

### Intrinsic Function
- CloudFormation에서 지원하는 기본 함수로 스택의 관리의 관리를 위한 다양한 기능을 제공
    - Resource 속성, Outputs, Metadata, Update Policy에서만 사용 가능
- 두 가지 호출 방식
    - Full Function
        - Fn::함수명
    - Short Form
        - !함수명
- 예 : Ref: 파라미터 혹은 리소스를 참조하기 위한 함수 -> "Ref : Logical ID" / !Ref Logical ID
- 주요 사용 사례
    - 다른 리소스 참조, AWS 계정의 AZ 목록 확보, 리소스의 속성 불러오기 등

### CloudFormation Pseudo Parameter
- CloudFormation에서 템플릿 프로비전 시 동적으로 넣어주는 파라미터
- 종류
    - AWS::AccountId : 리소스가 프로비전되는 계정의 Account ID
    - AWS::Region : 리소스가 프로비전되는 계정의 리전
    - AWS::StackName: 리소스가 프로비전되는 계정의 스택 이름
- 주요 사용 사례
    - 리소스의 이름 등에 계정명 혹은 리전명등을 포함하여 중복된 이름이 나오지 않도록 설정
    - 프로비전 시점에 다양한 리소스 조회
- !Ref 혹은 !Sub로 활용
    - !Sub : 특정 값들(파라미터, 다른 함수 등)로 스트링을 만드는 함수

### Mapping Section
- CloudFormation에서 미리 Map으로 데이터를 정의할 수 있는 섹션
    - 프로비전하는 상황에 따라 알맞은 값을 선택할 수 있도록 미리 데이터를 저장
    - 예 : 리전 별로 AMI ID를 미리 입력(매핑)
- Fn::FindInMap instrisic function으로 활용
    - Fn::FindInMap:[Mapname, TopLevelKey, SecondLevelKey] /
    !FindInMap[Mapname, TopLevelKey, SecondLevelKey]
- 주요 사용 사례
    - 리전 별 AMI 선택
    - Route53 Hosted Zone 선택 등

### Output Section
- 리소스가 프로비전 된 후에 필요한 정보를 스택 바깥으로 내보내는 섹션
    - 내보낸 정보는 다른 스택에서 참조하여 쓰거나 콘솔에서 확인 가능
- 구성
    - Logical ID
    - 설명
    - 값
    - 이름

### Condition Section
- 조건에 따라 리소스 생성의 의사결정 가능
    - 예 : Dev 리소스라면 더 작은 타입의 EC2 인스턴스
    - 예 : Prod 리소스라면 Multi-AZ로 DB 구성
- !If Instrinsic Function으로 활용